In [1]:
import pygame
import sys
from collections import defaultdict
import random

"""
All drawings related to pygame below
"""

# all set up
if True:
    # init pygame
    pygame.init()

    # init colors
    black = (0, 0, 0)
    white = (255, 255, 255)
    grey = (169,169,169)
    red = (255, 0, 0)
    blue = (0, 0, 255)
    green = (0, 255, 0)
    lRlue = (137, 207, 240)
    lRed = (255, 114, 111)
    dgrey = (100, 100, 100)

    displayWidth = 700
    displayHeight = 700

    gameDisplay = pygame.display.set_mode((displayWidth,displayHeight))
    pygame.display.set_caption('CHESS')
    font = pygame.font.SysFont("Arial", 15)
    clock = pygame.time.Clock()

    peices = {}

    peices["bh"] =  pygame.transform.scale(pygame.image.load('blackHorse.png'), (80, 80))
    peices["bb"] = pygame.transform.scale(pygame.image.load('blackBishop.png'), (80, 80))
    peices["br"] = pygame.transform.scale(pygame.image.load('blackRook.png'), (80, 80))
    peices["bq"] = pygame.transform.scale(pygame.image.load('blackQueen.png'), (80, 80))
    peices["bk"] = pygame.transform.scale(pygame.image.load('blackKing.png'), (80, 80))
    peices["bp"] = pygame.transform.scale(pygame.image.load('blackPawn.png'), (80, 80))

    peices["wh"] = pygame.transform.scale(pygame.image.load('whiteHorse.png'), (80, 80))
    peices["wb"] = pygame.transform.scale(pygame.image.load('whiteBishop.png'), (80, 80))
    peices["wr"] = pygame.transform.scale(pygame.image.load('whiteRook.png'), (80, 80))
    peices["wq"] = pygame.transform.scale(pygame.image.load('whiteQueen.png'), (80, 80))
    peices["wk"] = pygame.transform.scale(pygame.image.load('whiteKing.png'), (80, 80))
    peices["wp"] = pygame.transform.scale(pygame.image.load('whitePawn.png'), (80, 80))

    rowConversion = {"1":0, "2":1, "3":2, "4":3, "5":4, "6":5, "7":6, "8":7}
    rowConversionOpp = {x:y for y,x in rowConversion.items()}
    colConversion = {"a":0, "b":1, "c":2, "d":3, "e":4, "f":5, "g":6, "h":7}
    colConversionOpp = {x:y for y,x in colConversion.items()}

    peiceScores = {"p":10, "b":30, "h":30, "r":50, "q":90, "k":900}
    gameOver = 10000

# creating the border for the board
def border():
    pygame.draw.line(gameDisplay, white, (20, 20), (20, 680), 2)
    pygame.draw.line(gameDisplay, white, (20, 20), (680, 20), 2)
    pygame.draw.line(gameDisplay, white, (680, 20), (680, 680), 2)
    pygame.draw.line(gameDisplay, white, (20, 680), (680, 680), 2)
    half = 82.5
    start = 20
    labelsNum = ["1","2","3","4","5","6","7","8"]
    labelsLetter = ["A", "B", "C", "D", "E", "F", "G", "H"]
    font = pygame.font.SysFont("Arial", 15)
    # label across
    for i in range(8):
        text = font.render((labelsLetter[i]), True, white)
        gameDisplay.blit(text, (start+(half/2-2), 3))
        start += half
    # label downwards
    start = 20
    for i in range(8):
        text = font.render((labelsNum[i]), True, white)
        gameDisplay.blit(text, (4, start+(half/2-2)))
        start += half

# creating the checkers on the board
def checkers(selection):
    start = [-1,-1]
    for x in range(8):
        start[0] += 1
        start[1] = -1
        for i in range(8):
            start[1] += 1
            if x%2 == 0:
                if i%2 == 0:
                    color = white
                else:
                    color = grey
            else:
                if i%2 == 0:
                    color = grey
                else:
                    color = white
            pygame.draw.rect(gameDisplay, color, (start[1]*82.5 + 20, start[0]*82.5 + 20, 82.5, 82.5))
    if selection != ():
        pygame.draw.rect(gameDisplay, dgrey, (selection[0]*82.5 + 20, selection[1]*82.5 + 20, 82.5, 82.5))

# placing peices on the board
def placePeices(layout):
    start = [-1, -1]
    for x in range(8):
        start[0] += 1
        start[1] = -1
        for y in range(8):
            start[1] += 1
            peice = layout.loc[x][y]
            if peice != "--":
                gameDisplay.blit(peices[peice], (start[1]*82.5 + 20, start[0]*82.5 + 20, 82.5, 82.5))

# grade board function
def gradeBoard(board): #--> # corresponding to score of board
    score = 0
    for i in range(8): # row
        for j in range(8): # col
            if board.loc[i][j][0] == "b":
                score += peiceScores[board.loc[i][j][1]]
            elif board.loc[i][j][0] == "w":
                score -= peiceScores[board.loc[i][j][1]] # black is negative
    return score

# basic minmax function
def minmax(board, moves):
    if board.whiteMove:
        bool = -1
    else:
        bool = 1
    curScore = gameOver # worst possible score for black or white
    bestMove = None
    random.shuffle(moves)
    for move in moves:
        board.blitMove(move)
        oppMoves = board.getMove()
        oppScore = -gameOver
        for oppMove in oppMoves:
            board.blitMove(oppMove)
            score = -bool * gradeBoard(board)
            if board.checkMate:
                oppScore = -bool * gameOver
            elif score > oppScore:
                oppScore = score
            board.unblitMove()
        if oppScore < curScore:
            curScore = oppScore
            bestMove = move
        board.unblitMove()
    return bestMove

'''
Below introduces classes board, movePeice, etc
'''   

# creating board class
class board():

    def __init__(self):
        self.loc = [["br", "bh", "bb", "bq", "bk", "bb", "bh", "br"], 
        ["bp", "bp", "bp", "bp", "bp", "bp", "bp", "bp"],
        ["--", "--", "--", "--", "--", "--", "--", "--"],
        ["--", "--", "--", "--", "--", "--", "--", "--"],
        ["--", "--", "--", "--", "--", "--", "--", "--"],
        ["--", "--", "--", "--", "--", "--", "--", "--"],
        ["wp", "wp", "wp", "wp", "wp", "wp", "wp", "wp"],
        ["wr", "wh", "wb", "wq", "wk", "wb", "wh", "wr"]]
        self.whiteCap = []
        self.blackCap = []
        self.whiteScore = 0 # not used
        self.blackScore = 0 # not used
        self.moves = []
        self.whiteMove = True
        self.wkPos = (6,4)
        self.bkPos = (0,4)
        self.checkMate = False # Helpful for AI
    
    # physically makes the move
    def blitMove(self, move):
        self.loc[move.startRow][move.startCol] = "--"
        self.loc[move.endRow][move.endCol] = move.capPeice
        self.moves.append(move)
        if move.capPeice == "wk":
            self.wkPos = (move.endRow, move.endCol)
        elif move.capPeice == "bk":
            self.bkPos = (move.endRow, move.endCol)
        self.whiteMove = not self.whiteMove

    # physically un-makes the moves
    def unblitMove(self):
        if len(self.moves) != 0:
            move = self.moves.pop()
            self.loc[move.startRow][move.startCol] = move.capPeice
            self.loc[move.endRow][move.endCol] = move.deadPeice
            if move.capPeice == "wk":
                self.wkPos = (move.startRow, move.startCol)
            elif move.capPeice == "bk":
                self.bkPos = (move.startRow, move.startCol)
            self.whiteMove = not self.whiteMove

    # main function for grabbing all moves
    def getMove(self):
        moves = []
        for i in range(8): # row
            for j in range(8): # col
                toGo = self.loc[i][j][0]
                if (toGo == "w" and self.whiteMove) or (toGo == "b" and not self.whiteMove):
                    p = self.loc[i][j][1]
                    if p == "p":
                        self.getPawnMoves(i,j,moves)
                    elif p == "r":
                        self.getRookMoves(i,j,moves)
                    elif p == "h":
                        self.getHorseMoves(i,j,moves)
                    elif p == "b":
                        self.getBishopMoves(i,j,moves)
                    elif p == "q":
                        self.getQueenMoves(i,j,moves)
                    elif p == "k":
                        self.getKingMoves(i,j,moves)
        return moves
   
    # valid pawn moves
    def getPawnMoves(self, i, j, moves):
        if self.whiteMove:
            if self.loc[i-1][j] == "--":
                moves.append(movePeice((j,i),(j,i-1),self.loc))
                if 0 <= (i-2) <=7:
                    if (self.loc[i-2][j] == "--") and (i == 6):
                        moves.append(movePeice((j,i),(j,i-2),self.loc)) # row, col stored opposite from i, j order (oops)
            if j-1 >= 0:
                if self.loc[i-1][j-1] != "--" and self.loc[i-1][j-1][0] != "w": # dont allow for taking diagnol without opponent peice
                    moves.append(movePeice((j,i),(j-1,i-1),self.loc))
            if j+1 <= 7:
                if self.loc[i-1][j+1] != "--" and self.loc[i-1][j+1][0] != "w": # dont allow for taking diagnol without opponent peice
                    moves.append(movePeice((j,i),(j+1,i-1),self.loc))
        else:
            if self.loc[i+1][j] == "--":
                moves.append(movePeice((j,i),(j,i+1),self.loc))
                if 0 <= (i+2) <=7:
                    if (self.loc[i+2][j] == "--") and (i == 1):
                        moves.append(movePeice((j,i),(j,i+2),self.loc))
            if j-1 >= 0:
                if self.loc[i+1][j-1] != "--" and self.loc[i+1][j-1][0] != "b": # dont allow for taking diagnol without opponent peice
                    moves.append(movePeice((j,i),(j-1,i+1),self.loc))
            if j+1 <= 7:
                if self.loc[i+1][j+1] != "--" and self.loc[i+1][j+1][0] != "b": # dont allow for taking diagnol without opponent peice
                    moves.append(movePeice((j,i),(j+1,i+1),self.loc))

    # valid rook moves
    def getRookMoves(self, i, j, moves):
        if self.whiteMove:
            bool = "b"
        else:
            bool = "w"
        for u in range(1,8):
            if 0 <= (i-u) <=7:
                if self.loc[i-u][j] == "--":
                    moves.append(movePeice((j,i),(j,i-u),self.loc))
                elif self.loc[i-u][j][0] == bool:
                    moves.append(movePeice((j,i),(j,i-u),self.loc))
                    break
                else:
                    break
        for u in range(1,8):
            if 0 <= (i+u) <=7:
                if self.loc[i+u][j] == "--":
                    moves.append(movePeice((j,i),(j,i+u),self.loc))
                elif self.loc[i+u][j][0] == bool:
                    moves.append(movePeice((j,i),(j,i+u),self.loc))
                    break
                else:
                    break
        for u in range(1,8):
            if 0 <= (j-u) <=7:
                if self.loc[i][j-u] == "--":
                    moves.append(movePeice((j,i),(j-u,i),self.loc))
                elif self.loc[i][j-u][0] == bool:
                    moves.append(movePeice((j,i),(j-u,i),self.loc))
                    break
                else:
                    break
        for u in range(1,8):
            if 0 <= (j+u) <=7:
                if self.loc[i][j+u] == "--":
                    moves.append(movePeice((j,i),(j+u,i),self.loc))
                elif self.loc[i][j+u][0] == bool:
                    moves.append(movePeice((j,i),(j+u,i),self.loc))
                    break
                else:
                    break

    # valid horse moves
    def getHorseMoves(self, i, j, moves):
        if self.whiteMove:
            bool = "b"
        else:
            bool = "w"
        if 0 <= (i-2) <=7:
            if 0 <= (j-1) <=7:
                if self.loc[i-2][j-1][0] == bool or self.loc[i-2][j-1] == "--":
                    moves.append(movePeice((j,i),(j-1,i-2),self.loc))
            if 0 <= (j+1) <=7:
                if self.loc[i-2][j+1][0] == bool or self.loc[i-2][j+1] == "--":
                    moves.append(movePeice((j,i),(j+1,i-2),self.loc))
        
        if 0 <= (i+2) <=7:
            if 0 <= (j-1) <=7:
                if self.loc[i+2][j-1][0] == bool or self.loc[i+2][j-1] == "--":
                    moves.append(movePeice((j,i),(j-1,i+2),self.loc))
            if 0 <= (j+1) <=7:
                if self.loc[i+2][j+1][0] == bool or self.loc[i+2][j+1] == "--":
                    moves.append(movePeice((j,i),(j+1,i+2),self.loc))
        
        if 0 <= (j-2) <=7:
            if 0 <= (i-1) <=7:
                if self.loc[i-1][j-2][0] == bool or self.loc[i-1][j-2] == "--":
                    moves.append(movePeice((j,i),(j-2,i-1),self.loc))
            if 0 <= (i+1) <=7:
                if self.loc[i+1][j-2][0] == bool or self.loc[i+1][j-2] == "--":
                    moves.append(movePeice((j,i),(j-2,i+1),self.loc))
        
        if 0 <= (j+2) <=7:
            if 0 <= (i-1) <=7:
                if self.loc[i-1][j+2][0] == bool or self.loc[i-1][j+2] == "--":
                    moves.append(movePeice((j,i),(j+2,i-1),self.loc))
            if 0 <= (i+1) <=7:
                if self.loc[i+1][j+2][0] == bool or self.loc[i+1][j+2] == "--":
                    moves.append(movePeice((j,i),(j+2,i+1),self.loc))

    # valid bishop moves
    def getBishopMoves(self, i, j, moves):
        if self.whiteMove:
            bool = "b"
        else:
            bool = "w"
        for u in range(1,8):
            if 0 <= (i-u) <=7 and 0 <= (j-u) <=7:
                if self.loc[i-u][j-u] == "--":
                    moves.append(movePeice((j,i),(j-u,i-u),self.loc))
                elif self.loc[i-u][j-u][0] == bool:
                    moves.append(movePeice((j,i),(j-u,i-u),self.loc))
                    break
                else:
                    break
        for u in range(1,8):
            if 0 <= (i-u) <=7 and 0 <= (j+u) <=7:
                if self.loc[i-u][j+u] == "--":
                    moves.append(movePeice((j,i),(j+u,i-u),self.loc))
                elif self.loc[i-u][j+u][0] == bool:
                    moves.append(movePeice((j,i),(j+u,i-u),self.loc))
                    break
                else:
                    break
        for u in range(1,8):
            if 0 <= (i+u) <=7 and 0 <= (j-u) <=7:
                if self.loc[i+u][j-u] == "--":
                    moves.append(movePeice((j,i),(j-u,i+u),self.loc))
                elif self.loc[i+u][j-u][0] == bool:
                    moves.append(movePeice((j,i),(j-u,i+u),self.loc))
                    break
                else:
                    break
        for u in range(1,8):
            if 0 <= (i+u) <=7 and 0 <= (j+u) <=7:
                if self.loc[i+u][j+u] == "--":
                    moves.append(movePeice((j,i),(j+u,i+u),self.loc))
                elif self.loc[i+u][j+u][0] == bool:
                    moves.append(movePeice((j,i),(j+u,i+u),self.loc))
                    break
                else:
                    break

    # valid queen moves
    def getQueenMoves(self, i, j, moves):
        if self.whiteMove:
            bool = "b"
        else:
            bool = "w"

        # moves straight
        for u in range(1,8):
            if 0 <= (i-u) <=7:
                if self.loc[i-u][j] == "--":
                    moves.append(movePeice((j,i),(j,i-u),self.loc))
                elif self.loc[i-u][j][0] == bool:
                    moves.append(movePeice((j,i),(j,i-u),self.loc))
                    break
                else:
                    break
        for u in range(1,8):
            if 0 <= (i+u) <=7:
                if self.loc[i+u][j] == "--":
                    moves.append(movePeice((j,i),(j,i+u),self.loc))
                elif self.loc[i+u][j][0] == bool:
                    moves.append(movePeice((j,i),(j,i+u),self.loc))
                    break
                else:
                    break
        for u in range(1,8):
            if 0 <= (j-u) <=7:
                if self.loc[i][j-u] == "--":
                    moves.append(movePeice((j,i),(j-u,i),self.loc))
                elif self.loc[i][j-u][0] == bool:
                    moves.append(movePeice((j,i),(j-u,i),self.loc))
                    break
                else:
                    break
        for u in range(1,8):
            if 0 <= (j+u) <=7:
                if self.loc[i][j+u] == "--":
                    moves.append(movePeice((j,i),(j+u,i),self.loc))
                elif self.loc[i][j+u][0] == bool:
                    moves.append(movePeice((j,i),(j+u,i),self.loc))
                    break
                else:
                    break
        
        #moves diagnol
        for u in range(1,8):
            if 0 <= (i-u) <=7 and 0 <= (j-u) <=7:
                if self.loc[i-u][j-u] == "--":
                    moves.append(movePeice((j,i),(j-u,i-u),self.loc))
                elif self.loc[i-u][j-u][0] == bool:
                    moves.append(movePeice((j,i),(j-u,i-u),self.loc))
                    break
                else:
                    break
        for u in range(1,8):
            if 0 <= (i-u) <=7 and 0 <= (j+u) <=7:
                if self.loc[i-u][j+u] == "--":
                    moves.append(movePeice((j,i),(j+u,i-u),self.loc))
                elif self.loc[i-u][j+u][0] == bool:
                    moves.append(movePeice((j,i),(j+u,i-u),self.loc))
                    break
                else:
                    break
        for u in range(1,8):
            if 0 <= (i+u) <=7 and 0 <= (j-u) <=7:
                if self.loc[i+u][j-u] == "--":
                    moves.append(movePeice((j,i),(j-u,i+u),self.loc))
                elif self.loc[i+u][j-u][0] == bool:
                    moves.append(movePeice((j,i),(j-u,i+u),self.loc))
                    break
                else:
                    break
        for u in range(1,8):
            if 0 <= (i+u) <=7 and 0 <= (j+u) <=7:
                if self.loc[i+u][j+u] == "--":
                    moves.append(movePeice((j,i),(j+u,i+u),self.loc))
                elif self.loc[i+u][j+u][0] == bool:
                    moves.append(movePeice((j,i),(j+u,i+u),self.loc))
                    break
                else:
                    break

    # valud king moves
    def getKingMoves(self, i, j, moves):
        if self.whiteMove:
            bool = "b"
        else:
            bool = "w"

        # basic 1 square moves
        if 0 <= i-1 <= 7:
            if self.loc[i-1][j] == "--" or self.loc[i-1][j][0] == bool:
                moves.append(movePeice((j,i),(j,i-1),self.loc))
        if 0 <= i+1 <= 7:
            if self.loc[i+1][j] == "--" or self.loc[i+1][j][0] == bool:
                moves.append(movePeice((j,i),(j,i+1),self.loc))
        if 0 <= j-1 <= 7:
            if self.loc[i][j-1] == "--" or self.loc[i][j-1][0] == bool:
                moves.append(movePeice((j,i),(j-1,i),self.loc))
        if 0 <= j+1 <= 7:
            if self.loc[i][j+1] == "--" or self.loc[i][j+1][0] == bool:
                moves.append(movePeice((j,i),(j+1,i),self.loc))

        # diagnol 1 square
        if 0 <= i-1 <= 7 and 0 <= j-1 <= 7:
            if self.loc[i-1][j-1] == "--" or self.loc[i-1][j-1][0] == bool:
                moves.append(movePeice((j,i),(j-1,i-1),self.loc))
        if 0 <= i-1 <= 7 and 0 <= j+1 <= 7:
            if self.loc[i-1][j+1] == "--" or self.loc[i-1][j+1][0] == bool:
                moves.append(movePeice((j,i),(j+1,i-1),self.loc))
        if 0 <= i+1 <= 7 and 0 <= j-1 <= 7:
            if self.loc[i+1][j-1] == "--" or self.loc[i+1][j-1][0] == bool:
                moves.append(movePeice((j,i),(j-1,i+1),self.loc))
        if 0 <= i+1 <= 7 and 0 <= j+1 <= 7:
            if self.loc[i+1][j+1] == "--" or self.loc[i+1][j+1][0] == bool:
                moves.append(movePeice((j,i),(j+1,i+1),self.loc))

    # check if move results in check
    def checkCheck(self):
        moves = self.getMove()
        board = self.loc
        capW = self.whiteCap
        capB = self.blackCap
        moveList = self.moves
        wkPos = self.wkPos
        bkPos = self.bkPos
        for i in range(len(moves)-1, -1, -1):
            self.blitMove(moves[i])
            self.whiteMove = not self.whiteMove
            if self.whiteMove:
                x = self.target(self.wkPos[0], self.wkPos[1])
            else:
                x = self.target(self.bkPos[0], self.bkPos[1])
            if x:
                moves.remove(moves[i])
            self.whiteMove = not self.whiteMove
            self.unblitMove()
        if len(moves) == 0:
            if x:
                self.checkMate = True
        return moves

    # helper function for checkCheck
    def target(self, i, j):
        self.whiteMove = not self.whiteMove
        othermoves = self.getMove()
        self.whiteMove = not self.whiteMove
        for move in othermoves:
            if move.endRow == i and move.endCol == j:
                return True
        return False

    # checks pawns to see if they can be promoted
    def checkPawns(self):
        for i in range(8):
            if self.loc[1][i][1] == "p":
                self.loc[1][i] == "wq"
        for i in range(8):
            if self.loc[7][i][1] == "p":
                self.loc[7][i] == "wk"
        
# defining move class
class movePeice():

    def __init__(self, start, end, board):
        self.startRow = start[1]
        self.startCol = start[0]
        self.endRow = end[1]
        self.endCol = end[0]
        self.capPeice = board[self.startRow][self.startCol]
        self.deadPeice = board[self.endRow][self.endCol]
        self.eq = int(str(self.startRow) + str(self.startCol) + str(self.endRow) + str(self.endCol)) 

    def __eq__(self, other):
        if isinstance(other, movePeice):
            return self.eq == other.eq
        return False

'''
Creates main gameloop
'''

def gameLoop():
    selection = () # keeps track of the peice slection (x, y)
    clickLog = []
    x = board() # init board
    validMoves = x.checkCheck()
    newMove = False

    while True:

        for event in pygame.event.get(): # quit sequence
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                # grub position of mouse and click
                mouse = pygame.mouse.get_pos()
                mouseColumn = mouse[0]//82.5 # x pos of mouse (not adding 20)
                mouseRow = mouse[1]//82.5 # y pos of mouse (not adding 20)
                if selection == (int(mouseColumn), int(mouseRow)):
                    selection = ()
                    clickLog = []
                else:
                    selection = (int(mouseColumn), int(mouseRow))
                    clickLog.append(selection)
                if len(clickLog) == 2:
                    move = movePeice(clickLog[0], clickLog[1], x.loc)
                    if move in validMoves:
                        x.blitMove(move)
                        newMove = True
                    selection = ()
                    clickLog = []
        
        if newMove:
            validMoves = x.checkCheck()
            newMove = False
        if not x.whiteMove:
            pygame.time.wait(2)
            move = minmax(x, validMoves)
            x.blitMove(move)
            newMove = True
            selection = ()
            clickLog = []
        # init background
        gameDisplay.fill(black)
        # display checkers on the board
        checkers(selection)
        # display borders for chess board
        border()
        # display game peices in current state
        placePeices(x)
        # check pawns
        x.checkPawns()
        # blit captured peices
        
        # blit black screen if over
        if "bk" in x.whiteCap:
            gameDisplay.fill(black)
            font = pygame.font.SysFont("Arial", 50)
            gameDisplay.blit(font.render('WHITE WINS', True, white), (200, 325))
            gameDisplay.blit(font.render('WHITE WINS', True, grey), (202, 327))
        elif "wk" in x.blackCap:
            gameDisplay.fill(black)
            font = pygame.font.SysFont("Arial", 50)
            gameDisplay.blit(font.render('BLACK WINS', True, white), (200, 325))
            gameDisplay.blit(font.render('BLACK WINS', True, grey), (202, 327))
        pygame.display.flip()

gameLoop()
pygame.quit
quit()



pygame 2.5.1 (SDL 2.28.2, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 